In [5]:
import pandas as pd

### Airbnb front pages dataset
This dataset contains the information gathered from the rooms at all the search pages.
It contains:
- `roomTitle`: the type of the room.
- `roomPrice`: the price for a night.
- `roomURL`: the url for the room page, used for merging in the next phase.
- `hostType`: the class of the host.

In [6]:
airbnb_frontPage = pd.read_json('datasets/airbnb_frontPage.json', encoding='utf-8')
airbnb_frontPage.drop(columns= ["Keyword", "Host", "roomName", "roomRating", "roomReviewcount"], inplace=True)

airbnb_frontPage.head(2)

,roomTitle,roomPrice,roomURL,hostType
0,Loft em Campos do Jordão,R$268 por noite,https://www.airbnb.com/rooms/92836566916150808...,Preferido dos hóspedes\nPreferido dos hóspedes
1,Loft em Campos do Jordão,R$99 por noite,https://www.airbnb.com/rooms/53832612?adults=2...,Preferido dos hóspedes\nPreferido dos hóspedes


In [7]:
airbnb_frontPage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   roomTitle  672 non-null    object
 1   roomPrice  672 non-null    object
 2   roomURL    672 non-null    object
 3   hostType   672 non-null    object
dtypes: object(4)
memory usage: 21.1+ KB


### Airbnb room pages
This dataset contains some information scraped from the room's page.
- `Page_URL`: the url for the room page, used for merging in the next phase.
- `Rating`: the rating of the room.
- `Number_of_Reviews`: the number of reviews registered.
- `Amenities`: List of each rooms' amenities.

In [8]:
airbnb_rooms = pd.read_json('airbnb_rooms.json', encoding='utf-8')
airbnb_rooms.drop(columns= ["Title", "Location", "Number_of_Guests", "Number_of_Bedrooms", "Number_of_Beds", "Number_of_Bath", "Price", "Sleeping_Arrangements", "Hosted_by", "Response_Rate", "Image_1", "Image_2", "Image_3", "Current_Time"], inplace=True)
airbnb_rooms.head(2)

FileNotFoundError: File airbnb_rooms.json does not exist

## Merging the DataFrames

In [ ]:
merged_df = pd.merge(airbnb_frontPage, airbnb_rooms, left_on='roomURL', right_on='Page_URL')
df = merged_df.drop(columns=["Page_URL"]) # keeping the url for future cleaning
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   roomTitle          669 non-null    object
 1   roomPrice          669 non-null    object
 2   roomURL            669 non-null    object
 3   hostType           669 non-null    object
 4   Rating             669 non-null    object
 5   Number_of_Reviews  669 non-null    object
 6   Amenities          669 non-null    object
dtypes: object(7)
memory usage: 36.7+ KB


## Cleaning and Preprocessing

In [ ]:
# Renaming columns for consistency
df.rename(columns={
    'roomTitle' : 'roomType',
    'Rating' : 'rating',
    'Number_of_Reviews' : 'countReviews',
    'Amenities': 'amenities',
}, inplace=True)

In [ ]:
# Drop duplicate rows
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   roomType      669 non-null    object
 1   roomPrice     669 non-null    object
 2   roomURL       669 non-null    object
 3   hostType      669 non-null    object
 4   rating        669 non-null    object
 5   countReviews  669 non-null    object
 6   amenities     669 non-null    object
dtypes: object(7)
memory usage: 36.7+ KB


In [ ]:
# Fixing 'roomType' column
df['roomType'] = df['roomType'].str.split().str[0]  # Selecting the first word
df['roomType'] = df['roomType'].astype('category')  # Transforming in category dtype

df['roomType'].value_counts()  # Checking categories

roomType
Casa           113
Cabana         105
Apartamento    102
Quarto          89
Chalé           83
Loft            45
Microcasa       42
Hotel           21
Pousada         18
Lugar           13
Contêiner       12
Suíte           11
Condomínio      10
Trailer          2
Name: count, dtype: int64

In [ ]:
# Fixing 'roomPrice' column
df['roomPrice'] = df['roomPrice'].str.extract('(\d+)').astype(float)  # Keeping just the numerical values
df.head(1)

,roomType,roomPrice,roomURL,hostType,rating,countReviews,amenities
0,Loft,268.0,https://www.airbnb.com/rooms/92836566916150808...,Preferido dos hóspedes\nPreferido dos hóspedes,4.89,74,


In [ ]:
# Fixing the 'hostType' column
import unicodedata

def normalize_text(text):  # Function to normalize the strings
    text = unicodedata.normalize('NFKD', text)  # Normalize Unicode characters
    text = text.encode('ascii', 'ignore').decode('ascii')  # Remove non-ASCII characters
    return text.strip()

df['hostType'] = df['hostType'].apply(normalize_text)  # Apply normalization to the hostType column

print(df['hostType'].unique())  # Check unique values to debug

['Preferido dos hospedes\nPreferido dos hospedes' 'Superhost\nSuperhost'
 'De 18 a 20 de set.\n18  20 de set.' '']


In [ ]:
# Replace specific problematic strings
df['hostType'] = df['hostType'].replace({
    'Preferido dos hospedes\nPreferido dos hospedes': 'preferido',
    'Superhost\nSuperhost': 'superhost',
    'De 18 a 20 de set.\n18  20 de set.': None,
    '': 'no_class'
}).astype('category')

# Display value counts
print(df['hostType'].value_counts())

hostType
preferido    476
no_class     123
superhost     69
Name: count, dtype: int64


In [ ]:
# This code will process the DataFrame, aggregate the amenities, 
# and perform one-hot encoding to prepare the data for model training. 

df['amenities'] = df['amenities'].str.split('\n')

aggregation_map = {
    'wifi': 'WiFi',
    'hd': 'HDTV',
    'tv': 'TV',
    'netflix': 'Streaming Service',
    'prime': 'Streaming Service',
    'roku': 'Streaming Service',
    'disney+': 'Streaming Service',
    'hbo max': 'Streaming Service',
    'streaming': 'Streaming Service',
    'parking': 'Parking',
    'garage': 'Parking',
    'carport': 'Parking',
    'ac': 'Air Conditioning',
    'air conditioning': 'Air Conditioning',
    'pool': 'Pool',
    'hot tub': 'Bathtub',
    'sauna': 'Sauna',
    'fireplace': 'Fireplace',
    'microwave': 'Microwave',
    'washer': 'Washer',
    'dryer': 'Dryer',
    'refrigerator': 'Refrigerator',
    'smoke alarm': 'Smoke Alarm',
    'carbon monoxide alarm': 'Carbon Monoxide Alarm',
    'bathroom': 'Bathroom',
    'kitchen': 'Kitchen',
    'patio': 'Patio',
    'balcony': 'Balcony',
    'backyard': 'Backyard',
    'view': 'View',
    'security cameras': 'Security Cameras',
    'ev charger': 'EV Charger',
    'breakfast': 'Breakfast',
    'pets allowed': 'Pets Allowed',
    'luggage dropoff allowed': 'Luggage Dropoff Allowed',
    'step-free access': 'Accessible',
    'step-free path': 'Accessible',
    'step-free guest entrance': 'Accessible',
    'crib': 'Crib',
    'high chair': 'High Chair',
    'pack ’n play/travel crib': 'Crib',
    'bathtub': 'Bathtub'
}

# Function to aggregate amenities
def aggregate_amenity(amenity):
    for keyword, category in aggregation_map.items():
        if keyword.lower() in amenity.lower():
            return category
    return amenity

# Aggregate the amenities in the DataFrame
df['amenities'] = df['amenities'].apply(lambda amenities: [aggregate_amenity(amenity) for amenity in amenities])

# Flatten the list of amenities and get unique values
unique_amenities = set(amenity for amenities in df['amenities'] for amenity in amenities)

# Create separate columns for each amenity
for amenity in unique_amenities:
    df[amenity] = df['amenities'].apply(lambda x: 1 if amenity in x else 0)

# Drop the original amenities column
df = df.drop(columns=['amenities'])

In [ ]:
# Convert the DataFrame to CSV
df.to_csv('merged_df.csv', encoding='utf-8', sep=';')

## Fine cleaning

In [ ]:
# Load DataFrame 'dataset' and drop first column
dataset = pd.read_csv('merged_df.csv', encoding='utf-8', sep=';')
dataset.drop(dataset.columns[0], axis=1, inplace=True)
dataset.drop(dataset.columns[6], axis=1, inplace=True)

dataset.head(3)

,roomType,roomPrice,roomURL,hostType,rating,countReviews,Air Conditioning,TV,Dryer,Bathroom,...,Unavailable: Lock on bedroom door,Refrigerator,Smoking allowed,Patio,High Chair,Sauna,Crib,Washer,Accessible,Breakfast
0,Loft,268.0,https://www.airbnb.com/rooms/92836566916150808...,preferido,4.89,74.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Loft,99.0,https://www.airbnb.com/rooms/53832612?adults=2...,preferido,4.95,148.0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Chalé,329.0,https://www.airbnb.com/rooms/22588205?adults=2...,preferido,4.94,319.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   roomType                           666 non-null    object 
 1   roomPrice                          668 non-null    float64
 2   roomURL                            669 non-null    object 
 3   hostType                           668 non-null    object 
 4   rating                             637 non-null    float64
 5   countReviews                       637 non-null    float64
 6   Air Conditioning                   669 non-null    int64  
 7   TV                                 669 non-null    int64  
 8   Dryer                              669 non-null    int64  
 9   Bathroom                           669 non-null    int64  
 10  Ethernet connection                669 non-null    int64  
 11  Kitchen                            669 non-null    int64  

In [ ]:
# Finding registers with null values
nulls = dataset.isnull().sum()
nulls[nulls > 0]

## 'rating' and 'countReviews' columns have too much null values to judt drop
## or to fix by statistical replacements. I'll have to fix them manually...

roomType         3
roomPrice        1
hostType         1
rating          32
countReviews    32
dtype: int64

As I was trying to fix the null 'rating' and 'countReviews' manually, I've noticed that those are new rooms with no reviews or ratings. That means that roughly 5% of the rooms offered in my search were relatively new! 

That means we need to capture this 'newness' in new features.

In [ ]:
# Fill Null Values with Specific Indicators
dataset['rating'] = dataset['rating'].fillna(0)
dataset['countReviews'] = dataset['countReviews'].fillna(0)

# Create Additional Features
dataset['is_new'] = ((dataset['rating'] == 0) & (dataset['countReviews'] == 0)).astype(int)

# Fill remaining null values in other columns
for column in dataset.columns:
    if dataset[column].dtype in ['float64', 'int64']:
        dataset[column] = dataset[column].fillna(dataset[column].mean())
    else:
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])

In [ ]:
# Finding registers with null values
nulls_ = dataset.isnull().sum()
nulls_[nulls_ > 0]

Series([], dtype: int64)

In [ ]:
# There are some registries that have a price of just R$ 1.00, which doesn't make sense.
# Let's do a sanity check before going forward

wrong_price_registries = dataset[dataset['roomPrice'] <=80]
dataset.drop(wrong_price_registries.index, inplace=True)

print(dataset[dataset['roomPrice'] <= 80])

Empty DataFrame
Columns: [roomType, roomPrice, roomURL, hostType, rating, countReviews, Air Conditioning, TV, Dryer, Bathroom, Ethernet connection, Kitchen, Elevator, Luggage Dropoff Allowed, Smoke Alarm, WiFi, Parking, Pets Allowed, EV Charger, Bedroom, Lock on bedroom door, Fire pit, Lit path to the guest entrance, Waterfront, Long term stays allowed, Bathtub, Laundry room, Security Cameras, Baby bath, Pool, Microwave, HDTV, Bedroom 2, View, Carbon Monoxide Alarm, Unavailable: Lock on bedroom door, Refrigerator, Smoking allowed, Patio, High Chair, Sauna, Crib, Washer, Accessible, Breakfast, is_new]
Index: []

[0 rows x 46 columns]


In [ ]:
dataset.to_csv('final_dataset.csv', index=False)

print(dataset.info())

NameError: name 'dataset' is not defined